In [ ]:
# def register_patient(patient_id, personal_info, medical_history, insurance_info):
# def add_medical_staff(staff_id, name, specialization, shift_schedule, contact_info):
# def schedule_appointment(patient_id, doctor_id, appointment_date, appointment_type):
# def create_medical_record(patient_id, doctor_id, diagnosis, treatment, prescription):
# def process_billing(patient_id, services_list, insurance_coverage):
# def manage_emergency_admission(patient_id, emergency_type, severity_level):
# def track_medication_inventory(medication_id, quantity, expiry_date, supplier):
# def assign_room(patient_id, room_type, admission_date, expected_duration):
# def calculate_treatment_cost(patient_id, treatment_plan, insurance_details):
# def generate_patient_report(patient_id, report_type):
# def analyze_hospital_efficiency(metrics_type, time_period):
# def manage_discharge_process(patient_id, discharge_date, follow_up_instructions):

In [1]:
hospital_data = {
    'patients' : {},
    'medical_staff' : {},
}

def register_patient(patient_id, personal_info, medical_history, insurance_info):
    if patient_id in hospital_data['patients']:
        print("Patient already Registered")
        return
    hospital_data['patients'][patient_id] = {
        'personal_info': personal_info,
        'medical_history': medical_history,
        'insurance_info': insurance_info,   
    }
    print(f"Patient {personal_info['name']} Registered Successfully.")

def add_medical_staff(staff_id, name, specialization, shift_schedule, contact_info):
    if staff_id in hospital_data['medical_staff']:
        print("Staff already Registered.")
        return
    hospital_data['medical_staff'][staff_id] = {
        'name':name,
        'specialization' : specialization,
        'shift_schedule' : shift_schedule,
        'contact_info' : contact_info,
    }
    print(f"Medical staff {name} added.")


register_patient('P12345', {'name': 'Rajesh Kumar', 'age': 45, 'gender': 'Male', 'blood_type': 'B+'}, ['Hypertension'], {'provider': 'Star Health', 'policy': 'SH789456'})
add_medical_staff('D100', 'Dr. Priya Mehta', 'Cardiology', [('08:00', '16:00')], {'phone': '9876543210'})

Patient Rajesh Kumar Registered Successfully.
Medical staff Dr. Priya Mehta added.
